In [4]:
using CSV
using DataFrames
using Statistics
using MultivariateStats
using Distances
using LinearAlgebra

Cf. Python: Wall time: 2h 16min 21s(136 min.) 100 face >> 35 sec.
calcs01 19.4 sec. / 19.3 sec.
calcs02 18.9 sec. / 18.8 sec. (modi.) 

In [145]:
function calcs01(iter)
    path_home="C:/Usees/KANKI/Documents/github/habmap"
    path_file_face="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/faces_dataset.csv"
    path_file_v5="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_dataset.csv"
    path_file_v1="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_dataset_raw.csv"
    table_f=CSV.read(path_file_face,header=false, DataFrame)
    table_v5=CSV.read(path_file_v5,header=false, DataFrame)
    table_v1=CSV.read(path_file_v1,header=false, DataFrame)
    
    table_f=convert(Matrix,table_f)
    table_v5=convert(Matrix,table_v5)
    table_v1=convert(Matrix,table_v1)

    row, col=size(table_f)
    row_v1, col_v1=size(table_v1)
    println(row, " " , row_v1)
    
    g=zeros(Float16,row_v1,3)  
    terrain_variables=zeros(Float16, row, 12)

    bottom_all=findmin(table_v5[:,3])[1]    
    for c=1:row
        for d=1:3
            g[c,d]=(table_v5[table_f[c,1]+1,d]+table_v5[table_f[c,2]+1,d]+table_v5[table_f[c,3]+1,d])/3
        end
    end
    
    for c=1:row
        terrain_variables[c,1]=g[c,3]
        terrain_variables[c,2]=g[c,3]-bottom_all
    end
    
    for c=1:iter
        #index=findall(g[c,1]-0.8 .< table_v1[:,1] .< g[c,1]+0.8)
        #println(size(index)[1]/row_v1)
        distance_plane=zeros(Float32, row_v1, 1)
        distance=zeros(Float32, row_v1, 1)
        ver_in_ker4=Vector{Float64}(undef, 3)
        ver_in_ker3=Vector{Float64}(undef, 3)
        ver_in_ker2=Vector{Float64}(undef, 3)
        ver_in_ker1=Vector{Float64}(undef, 3)

        for i2=1:row_v1    
            distance_plane[i2,1]=(table_v1[i2,1]-g[c,1])^2+(table_v1[i2,2]-g[c,2])^2
            distance[i2,1]=distance_plane[i2,1]+(table_v1[i2,3]-g[c,3])^2
        end
        
        index_p4=findall(distance_plane .< 0.64)
        index_p3=findall(distance_plane .< 0.16)
        index_p2=findall(distance_plane .< 0.04)
        index_p1=findall(distance_plane .< 0.01)
        index4=findall(distance .< 0.64)
        index3=findall(distance .< 0.16)
        index2=findall(distance .< 0.04)
        index1=findall(distance .< 0.01)
        
        ver_in_ker4=zeros(Float32, size(index4)[1], 3)
        ver_in_ker3=zeros(Float32, size(index3)[1], 3)
        ver_in_ker2=zeros(Float32, size(index2)[1], 3)
        ver_in_ker1=zeros(Float32, size(index1)[1], 3)
        ver_in_ker_p4=zeros(Float32, size(index_p4)[1], 3)
        ver_in_ker_p3=zeros(Float32, size(index_p3)[1], 3)
        ver_in_ker_p2=zeros(Float32, size(index_p2)[1], 3)
        ver_in_ker_p1=zeros(Float32, size(index_p1)[1], 3)        
        
        count=1
        for index in index4
            ver_in_ker4[count,1:3]=table_v1[index[1],1:3]
            count+=1
        end
        count=1
        for index in index3
            ver_in_ker3[count,1:3]=table_v1[index[1],1:3]
            count+=1
        end
        count=1
        for index in index2
            ver_in_ker2[count,1:3]=table_v1[index[1],1:3]
            count+=1
        end
        count=1
        for index in index1
            ver_in_ker1[count,1:3]=table_v1[index[1],1:3]
            count+=1
        end
        
        count=1
        for index in index_p4
            ver_in_ker_p4[count,1:3]=table_v1[index[1],1:3]
            count+=1
        end
        count=1
        for index in index_p3
            ver_in_ker_p3[count,1:3]=table_v1[index[1],1:3]
            count+=1
        end
        count=1
        for index in index_p2
            ver_in_ker_p2[count,1:3]=table_v1[index[1],1:3]
            count+=1
        end
        count=1
        for index in index_p1
            ver_in_ker_p1[count,1:3]=table_v1[index[1],1:3]
            count+=1
        end
        
        bottom1=findmin(ver_in_ker_p1[:,3])[1]
        bottom2=findmin(ver_in_ker_p2[:,3])[1]
        bottom3=findmin(ver_in_ker_p3[:,3])[1]
        bottom4=findmin(ver_in_ker_p4[:,3])[1]
        terrain_variables[c,3]=g[c,3]-bottom1
        terrain_variables[c,4]=g[c,3]-bottom2
        terrain_variables[c,5]=g[c,3]-bottom3
        terrain_variables[c,6]=g[c,3]-bottom4
        meanz1=Statistics.mean(ver_in_ker_p1[:,3])
        meanz2=Statistics.mean(ver_in_ker_p2[:,3])
        meanz3=Statistics.mean(ver_in_ker_p3[:,3])
        meanz4=Statistics.mean(ver_in_ker_p4[:,3])
        terrain_variables[c,7]=g[c,3]-meanz1        
        terrain_variables[c,8]=g[c,3]-meanz2
        terrain_variables[c,9]=g[c,3]-meanz3
        terrain_variables[c,10]=g[c,3]-meanz4
        M4=fit(PCA,ver_in_ker4)
        M3=fit(PCA,ver_in_ker3)
        M2=fit(PCA,ver_in_ker2)
        M1=fit(PCA,ver_in_ker1)

    end
    return terrain_variables
end

@time calcs01(100)

23077 310354
 26.897974 seconds (644.57 M allocations: 10.965 GiB, 7.48% gc time)


23077×12 Array{Float16,2}:
 -1.176   0.7563  0.646    0.6963  0.7236  …  -0.1681    -0.2727   0.0  0.0
 -0.7466  1.186   0.0737   0.1735  0.4282      0.05215   -0.1268   0.0  0.0
 -0.756   1.176   0.0723   0.1642  0.4236      0.04898   -0.1387   0.0  0.0
 -0.739   1.193   0.05783  0.1494  0.427       0.04736   -0.11316  0.0  0.0
 -0.7583  1.174   0.07355  0.1617  0.4216      0.03244   -0.141    0.0  0.0
 -0.762   1.17    0.07623  0.1693  0.4177  …   0.01081   -0.1434   0.0  0.0
 -0.752   1.18    0.07635  0.1681  0.423       0.0514    -0.1351   0.0  0.0
 -0.722   1.21    0.0596   0.1484  0.4177      0.04523   -0.0928   0.0  0.0
 -0.741   1.19    0.05463  0.147   0.4246      0.03952   -0.11285  0.0  0.0
 -0.7524  1.18    0.06805  0.1749  0.4219      0.00896   -0.1273   0.0  0.0
 -0.755   1.177   0.0763   0.1725  0.4194  …  -0.002615  -0.129    0.0  0.0
 -0.778   1.154   0.0813   0.1693  0.402      -0.01852   -0.1627   0.0  0.0
 -0.759   1.173   0.07294  0.1613  0.4211      0.05353   -0.1

In [146]:
function calcs02(iter::Int8)
    path_home="C:/Usees/KANKI/Documents/github/habmap"
    path_file_face="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/faces_dataset.csv"
    path_file_v5="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_dataset.csv"
    path_file_v1="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_dataset_raw.csv"
    table_f=CSV.read(path_file_face,header=false, DataFrame)
    table_v5=CSV.read(path_file_v5,header=false, DataFrame)
    table_v1=CSV.read(path_file_v1,header=false, DataFrame)
    
    table_f=convert(Matrix,table_f)
    table_v5=convert(Matrix,table_v5)
    table_v1=convert(Matrix,table_v1)

    row, col=size(table_f)
    row_v1, col_v1=size(table_v1)
    
    g=zeros(Float16,row_v1,3)  
    terrain_variables=zeros(Float16, row, 12)

    bottom_all=findmin(table_v5[:,3])[1]    
    for c=1:row
        for d=1:3
            g[c,d]=(table_v5[table_f[c,1]+1,d]+table_v5[table_f[c,2]+1,d]+table_v5[table_f[c,3]+1,d])/3
        end
    end
    
    for c=1:row
        terrain_variables[c,1]=g[c,3]
        terrain_variables[c,2]=g[c,3]-bottom_all
    end
    
    for c=1:iter
        distance=zeros(Float32, row_v1, 2)
        for i2=1:row_v1
            distance[i2,1]=(table_v1[i2,1]-g[c,1])^2+(table_v1[i2,2]-g[c,2])^2
            distance[i2,2]=distance[i2,1]+(table_v1[i2,3]-g[c,3])^2
        end
        
        kernels=[0.64, 0.16, 0.04, 0.01]
        
        #Calculating rugosity and bpi
        
        for (countk, kernel) in enumerate(kernels)
            index_p=findall(distance[:,1] .< kernel)
            ver_in_ker_p=zeros(Float32, size(index_p)[1], 1)
            for (i, index) in enumerate(index_p)
                ver_in_ker_p[i,1]=table_v1[index[1],3]
            end 
            
            index_v=findall(distance[:,2] .< kernel)
            ver_in_ker=zeros(Float32, size(index_v)[1], 3)
            for (i, index) in enumerate(index_v)
                ver_in_ker[i,1:3]=table_v1[index[1],1:3]
            end
            
            bottom=findmin(ver_in_ker_p)[1]
            meanz=Statistics.mean(ver_in_ker_p[:,1])
            terrain_variables[c,countk+2]=g[c,3]-bottom
            terrain_variables[c,countk+6]=g[c,3]-meanz
            M=fit(PCA,ver_in_ker)
        end
    end
    return terrain_variables
end

@time calcs02(10)

  2.691318 seconds (65.59 M allocations: 1.175 GiB, 7.28% gc time)


23077×12 Array{Float16,2}:
 -1.176   0.7563  0.7563  0.7236  0.6963  …  -0.002792   0.03075    0.0  0.0
 -0.7466  1.186   0.5474  0.4282  0.1735      0.01471   -3.1e-5     0.0  0.0
 -0.756   1.176   0.5283  0.4236  0.1642      0.01066   -0.00331    0.0  0.0
 -0.739   1.193   0.5454  0.427   0.1494      0.01089   -0.006565   0.0  0.0
 -0.7583  1.174   0.54    0.4216  0.1617      0.01051   -0.002016   0.0  0.0
 -0.762   1.17    0.5396  0.4177  0.1693  …   0.011154  -0.00165    0.0  0.0
 -0.752   1.18    0.532   0.423   0.1681      0.01309   -0.00255    0.0  0.0
 -0.722   1.21    0.5547  0.4177  0.1484      0.01654   -0.00458    0.0  0.0
 -0.741   1.19    0.5566  0.4246  0.147       0.00631   -0.00894    0.0  0.0
 -0.7524  1.18    0.556   0.4219  0.1749      0.01291   -0.0008903  0.0  0.0
 -0.755   1.177   0.0     0.0     0.0     …   0.0        0.0        0.0  0.0
 -0.778   1.154   0.0     0.0     0.0         0.0        0.0        0.0  0.0
 -0.759   1.173   0.0     0.0     0.0         0.0

In [9]:
path_home="C:/Usees/KANKI/Documents/github/habmap"
path_file_face="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/faces_dataset.csv"
path_file_v5="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_dataset.csv"
path_file_v1="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_dataset_raw.csv"
path_file_n5="C:/Users/KANKI/Documents/github/habmap/Sfm_model/modelA01/ply_parts/vertice_normal.csv"

table_f=CSV.read(path_file_face,header=false, DataFrame)
table_v5=CSV.read(path_file_v5,header=false, DataFrame)
table_v1=CSV.read(path_file_v1,header=false, DataFrame)
table_n5=CSV.read(path_file_n5,header=false, DataFrame)

table_f=convert(Matrix,table_f)
table_v5=convert(Matrix,table_v5)
table_v1=convert(Matrix,table_v1)
table_n5=convert(Matrix, table_n5)
row=size(table_f)[1]


function calcs03B(table_f, table_v5, table_v1, iter)
    row, col=size(table_f)
    row_v1, col_v1=size(table_v1)
    metav=projection(fit(PCA,table_v1'))
    tilt_direction=180+sign(metav[2])*(90+90-acos(metav[1]/(metav[1]^2+metav[2]^2)^0.5)/pi*180)
    g=zeros(Float32,row_v1,3)
    ng=zeros(Float32,row_v1,3)
    terrain_variables=zeros(Float32, row, 35)

    bottom_all=findmin(table_v5[:,3])[1]    
    for c=1:row
        for d=1:3
            g[c,d]=(table_v5[table_f[c,1]+1,d]+table_v5[table_f[c,2]+1,d]+table_v5[table_f[c,3]+1,d])/3
            ng[c,d]=(table_n5[table_f[c,1]+1,d]+table_n5[table_f[c,2]+1,d]+table_n5[table_f[c,3]+1,d])/3
        end
    end

    for c=1:row
        terrain_variables[c,1]=g[c,3]
        terrain_variables[c,2]=g[c,3]-bottom_all
    end
    
    for c=1:iter
        distance=zeros(Float32, row_v1, 2)
        for i2=1:row_v1
            distance[i2,1]=(table_v1[i2,1]-g[c,1])^2+(table_v1[i2,2]-g[c,2])^2
            distance[i2,2]=distance[i2,1]+(table_v1[i2,3]-g[c,3])^2
        end
        
        kernels=[0.01, 0.04, 0.16, 0.64]
        
        #Calculating rugosity and bpi
        
        for (countk, kernel) in enumerate(kernels)
            index_p=findall(distance[:,1] .< kernel)
            ver_in_ker_p=zeros(Float32, size(index_p)[1], 1)
            for (i, index) in enumerate(index_p)
                ver_in_ker_p[i,1]=table_v1[index[1],3]
            end 
            
            index_v=findall(distance[:,2] .< kernel)
            ver_in_ker=zeros(Float32, size(index_v)[1], 3)
            for (i, index) in enumerate(index_v)
                ver_in_ker[i,1:3]=table_v1[index[1],1:3]
            end
            
            bottom=findmin(ver_in_ker_p[:,1])[1]
            meanz=Statistics.mean(ver_in_ker_p[:,1])
            terrain_variables[c,countk+2]=g[c,3]-bottom
            terrain_variables[c,countk+6]=g[c,3]-meanz
            M=fit(PCA,ver_in_ker')
            if size(projection(M))[1] > 3
                println(size(projection(M)))
            end
            
            if size(projection(M))[2]==2
                v3=cross(projection(M)[:,1],projection(M)[:,2])
            elseif size(projection(M))[2]==3
                v3=projection(M)[:,3]
            else
                println(size(projection(M)))
            end
            v3=v3*sign(ng[c,3])*sign(v3[3])
            terrain_variables[c,countk+10]=90-acos(v3[1]/(v3[1]^2+v3[2]^2)^0.5)/pi*180
            terrain_variables[c,countk+14]=90-acos(v3[2]/(v3[1]^2+v3[2]^2)^0.5)/pi*180
            terrain_variables[c,countk+18]=180+sign(v3[2])*(90+terrain_variables[c,countk+10])
            terrain_variables[c,countk+22]=findmin([abs(tilt_direction-terrain_variables[c,countk+18]),
                    360-abs(tilt_direction-terrain_variables[c,countk+18])])[1]
            terrain_variables[c,countk+26]=((pi/2)-atan(v3[3]/((v3[1]^2+v3[2]^2)^0.5)))/pi*180

            distances=zeros(Float32, size(index_v)[1], 1)
            d=-dot(v3, [mean(ver_in_ker[:,1]), mean(ver_in_ker[:,2]), mean(ver_in_ker[:,3])])
            for j in 1:size(index_v)[1]
                distance[j,1]=(dot(v3, ver_in_ker[j,1:3])+d)/sqrt(dot(v3,v3))
            end
            terrain_variables[c,countk+30]=mean(distance)
        end
    end
    return terrain_variables
end

@time terrain_variables=calcs03B(table_f, table_v5, table_v1, 100)

  2.165228 seconds (7.61 M allocations: 2.114 GiB, 13.20% gc time)

23077×35 Array{Float32,2}:
 -1.17563   0.756259  0.645916   0.696403  …  4.16292  4.12406  3.97957  0.0
 -0.746547  1.18534   0.0737836  1.11937      4.89325  4.86727  4.74132  0.0
 -0.755625  1.17627   0.0725781  1.11029      4.86615  4.84067  4.71473  0.0
 -0.738991  1.1929    0.0575961  1.12693      4.96933  4.94168  4.81966  0.0
 -0.758527  1.17336   0.0733508  1.10739      4.83708  4.81173  4.68333  0.0
 -0.762139  1.16975   0.0762982  1.10378   …  4.80958  4.7837   4.65352  0.0
 -0.752011  1.17988   0.0763152  1.11391      4.89102  4.86527  4.74157  0.0
 -0.722222  1.20967   0.0595359  1.1437       5.06478  5.03602  4.91963  0.0
 -0.741074  1.19082   0.0547494  1.12484      4.95852  4.92998  4.80658  0.0
 -0.752242  1.17965   0.0682678  1.11368      4.85581  4.82795  4.69929  0.0
 -0.754886  1.177     0.0762733  1.11103   …  4.82935  4.80076  4.67095  0.0
 -0.777839  1.15405   0.0812641  1.08808      4.74224  4.71641  4.58402  0.0
 -0.758571  1.17332   0.073163   1.10735      4.8

In [225]:
table_v1=CSV.read(path_file_v1,header=false, DataFrame)
table_v1=convert(Matrix,table_v1)
M=fit(PCA,table_v1')

PCA(indim = 3, outdim = 3, principalratio = 1.0)

In [304]:
mean(table_v1)

-0.9645507207854155

In [279]:
table_v1[end,:]*(-1)

3-element Array{Float64,1}:
 1.828483
 2.10013
 0.996489

In [227]:
projection(M)

3×3 Array{Float64,2}:
 0.355061    0.91208   -0.20504
 0.933308   -0.35841    0.0218613
 0.0535493   0.199128   0.978509

In [255]:
vec=[
0 0 0
1 0 2
0.1 0 0.2
2 0 4
2 1 4
3 2 6
3 1 6
1 1 2
1 3 2
3 2 6
]

10×3 Array{Float64,2}:
 0.0  0.0  0.0
 1.0  0.0  2.0
 0.1  0.0  0.2
 2.0  0.0  4.0
 2.0  1.0  4.0
 3.0  2.0  6.0
 3.0  1.0  6.0
 1.0  1.0  2.0
 1.0  3.0  2.0
 3.0  2.0  6.0

In [256]:
M=fit(PCA, vec')

PCA(indim = 3, outdim = 2, principalratio = 1.0)

In [260]:
projection(M)[:,1]

3-element Array{Float64,1}:
 0.4378581137781208
 0.2034732439249236
 0.8757162275562413

In [261]:
v3=cross(projection(M)[:,1], projection(M)[:,2])

3-element Array{Float64,1}:
  0.8944271909999157
  4.163336342344337e-17
 -0.447213595499958

In [238]:
using LinearAlgebra

In [235]:
import Base

In [224]:
0.268*0.511+0.3575*(-0.8399)+0.894476*0.182334

-0.00022286301599994118

In [248]:
90-acos(v3[1]/(v3[1]^2+v3[2]^2)^0.5)/pi*180

-15.051828879749038